In [ ]:
%use adventOfCode

In [ ]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 1)

In [5]:
aoc.viewPartOne()

--- Day 1: Secret Entrance --- 
 The Elves have good news and bad news. 
 The good news is that they've discovered project management ! This has given them the tools they need to prevent their usual Christmas emergency. For example, they now know that the North Pole decorations need to be finished soon so that other critical tasks can start on time. 
 The bad news is that they've realized they have a different emergency: according to their resource planning, none of them have any time left to decorate the North Pole! 
 To save Christmas, the Elves need you to finish decorating the North Pole by December 12th . 
 Collect stars by solving puzzles. Two puzzles will be made available on each day; the second puzzle is unlocked when you complete the first. Each puzzle grants one star . Good luck! 
 You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the password seems to have been changed, so you can't get in. A document taped to the wall helpfully explains: 
 "Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination." 
 The safe has a dial with only an arrow on it; around the dial are the numbers 0 through 99 in order. As you turn the dial, it makes a small click noise as it reaches each number. 
 The attached document (your puzzle input) contains a sequence of rotations , one per line, which tell you how to open the safe. A rotation starts with an L or R which indicates whether the rotation should be to the left (toward lower numbers) or to the right (toward higher numbers). Then, the rotation has a distance value which indicates how many clicks the dial should be rotated in that direction. 
 So, if the dial were pointing at 11 , a rotation of R8 would cause the dial to point at 19 . After that, a rotation of L19 would cause it to point at 0 . 
 Because the dial is a circle, turning the dial left from 0 one click makes it point at 99 . Similarly, turning the dial right from 99 one click makes it point at 0 . 
 So, if the dial were pointing at 5 , a rotation of L10 would cause it to point at 95 . After that, a rotation of R5 could cause it to point at 0 . 
 The dial starts by pointing at 50 . 
 You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is the number of times the dial is left pointing at 0 after any rotation in the sequence . 
 For example, suppose the attached document contained the following rotations: 
 L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
 
 Following these rotations would cause the dial to move as follows: 
 
 The dial starts by pointing at 50 . 
 The dial is rotated L68 to point at 82 . 
 The dial is rotated L30 to point at 52 . 
 The dial is rotated R48 to point at 0 . 
 The dial is rotated L5 to point at 95 . 
 The dial is rotated R60 to point at 55 . 
 The dial is rotated L55 to point at 0 . 
 The dial is rotated L1 to point at 99 . 
 The dial is rotated L99 to point at 0 . 
 The dial is rotated R14 to point at 14 . 
 The dial is rotated L82 to point at 32 . 
 
 Because the dial points at 0 a total of three times during this process, the password in this example is 3 . 
 Analyze the rotations in your attached document. What's the actual password to open the door?

In [12]:
val exampleInput = """L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
""".trimIndent()
exampleInput

L68
L30
R48
L5
R60
L55
L1
L99
R14
L82

In [20]:
enum class Direction(val modifier: Int) { LEFT(-1), RIGHT(+1) }
data class Rotation(val direction: Direction, val distance: Int) {
    val movement = direction.modifier * distance

    constructor(rotationLine: String) : this(
        direction = when (rotationLine.first()) {
            'L' -> Direction.LEFT
            'R' -> Direction.RIGHT
            else -> error("Invalid rotation direction: ${rotationLine.first()}")
        },
        rotationLine.drop(1).toInt(),
    )
}

fun Int.rotate(rotation: Rotation) = (this + rotation.movement).mod(100)

fun Iterable<Rotation>.countZeroes(start: Int = 50) = runningFold(start) { position, rotation -> position.rotate(rotation) }.count { it == 0 }

In [21]:
val rotations = exampleInput.lines().map(::Rotation)
rotations.countZeroes()

3

In [24]:
val input = aoc.input().trimIndent().lines().map(::Rotation)
input.countZeroes()

1145

In [25]:
// aoc.submitPartOne(input.countZeroes())

Your answer: 1145. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [26]:
aoc.viewPartTwo()

--- Part Two --- 
 You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think. 
 As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow: 
 "Due to newer security protocols, please use password method 0x434C49434B until further notice." 
 You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times any click causes the dial to point at 0 , regardless of whether it happens during a rotation or at the end of one. 
 Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations: 
 
 The dial starts by pointing at 50 . 
 The dial is rotated L68 to point at 82 ; during this rotation, it points at 0 once . 
 The dial is rotated L30 to point at 52 . 
 The dial is rotated R48 to point at 0 . 
 The dial is rotated L5 to point at 95 . 
 The dial is rotated R60 to point at 55 ; during this rotation, it points at 0 once . 
 The dial is rotated L55 to point at 0 . 
 The dial is rotated L1 to point at 99 . 
 The dial is rotated L99 to point at 0 . 
 The dial is rotated R14 to point at 14 . 
 The dial is rotated L82 to point at 32 ; during this rotation, it points at 0 once . 
 
 In this example, the dial points at 0 three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be 6 . 
 Be careful: if the dial were pointing at 50 , a single rotation like R1000 would cause the dial to point at 0 ten times before returning back to 50 ! 
 Using password method 0x434C49434B, what is the password to open the door?

In [118]:
data class RotationResult(val position: Int, val numberOfZeroPasses: Int) {
    override fun toString() = "\n(pos=$position, zeroes=$numberOfZeroPasses)"
}

fun Int.rotateCountingZeroPasses(rotation: Rotation): RotationResult {
    val rawNewPosition = this + rotation.movement
    val (lower, upper) = listOf(this, rawNewPosition).sorted()
    // not the most efficient, but it works
    var numberOfZeroPasses = (lower..upper).count { it.mod(100) == 0 }
    // let's not count zeroes twice. If this == 0 it has already been counted
    if (this == 0) numberOfZeroPasses--
    val newPosition = rawNewPosition.mod(100)
    return RotationResult(newPosition, numberOfZeroPasses)
}

fun Iterable<Rotation>.countZeroPasses(start: Int = 50) =
    fold(RotationResult(start, 0)) { state, rotation ->
        val (newPos, zeroPasses) = state.position.rotateCountingZeroPasses(rotation)
        RotationResult(newPos, state.numberOfZeroPasses + zeroPasses)
    }.numberOfZeroPasses

fun Iterable<Rotation>.countingZeroPasses(start: Int = 50) =
    runningFold(RotationResult(start, 0)) { state, rotation ->
        val (newPos, zeroPasses) = state.position.rotateCountingZeroPasses(rotation)
        RotationResult(newPos, state.numberOfZeroPasses + zeroPasses)
    }

In [119]:
val rotations = exampleInput.lines().map(::Rotation)
DISPLAY(rotations)
rotations.countingZeroPasses()

[Rotation(direction=LEFT, distance=68), Rotation(direction=LEFT, distance=30), Rotation(direction=RIGHT, distance=48), Rotation(direction=LEFT, distance=5), Rotation(direction=RIGHT, distance=60), Rotation(direction=LEFT, distance=55), Rotation(direction=LEFT, distance=1), Rotation(direction=LEFT, distance=99), Rotation(direction=RIGHT, distance=14), Rotation(direction=LEFT, distance=82)]

[
(pos=50, zeroes=0), 
(pos=82, zeroes=1), 
(pos=52, zeroes=1), 
(pos=0, zeroes=2), 
(pos=95, zeroes=2), 
(pos=55, zeroes=3), 
(pos=0, zeroes=4), 
(pos=99, zeroes=4), 
(pos=0, zeroes=5), 
(pos=14, zeroes=5), 
(pos=32, zeroes=6)]

In [120]:
val input = aoc.input().trimIndent().lines().map(::Rotation)
input.countZeroPasses()

6561

In [122]:
aoc.submitPartTwo(input.countZeroPasses())

Your answer: 6561. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 1! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .